In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# cleaning bengal or english
!pip install git+https://github.com/csebuetnlp/normalizer

# kenlm for ctcdecode
!pip install https://github.com/kpu/kenlm/archive/master.zip

!pip install -r /content/drive/MyDrive/audio/whisper/requirements.txt
# 이거 안하면 저장할 때 오류남~
!pip uninstall peft -y
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-yffsxdnd
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-yffsxdnd
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6857 sha256=8ead03d8baab7378b517522d648068634ceaad344016905ddc85c8e641704939
  Stored in directory: /tmp/pip-ephem-wheel-cache-vokniw95/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186457 sha256=be9350c6569be02799eedb8747cb6dc3d1d3bbc

In [3]:
import os
import gc
import glob
import sys
from pathlib import Path


import pandas as pd
import gc
import numpy as np
from tqdm.notebook import tqdm

import librosa
import warnings
warnings.filterwarnings('ignore')
sys.path.append("/content/drive/MyDrive/audio/whisper/")

In [4]:
sentences = pd.read_csv("/content/drive/MyDrive/audio/dataset/normalized.csv")
indexes = set(pd.read_csv("/content/drive/MyDrive/audio/dataset/indexes.csv/indexes.csv")['id'])
print(len(sentences))

# mozilla-foundation/common_voice_11_0 dataset은 제거
sentences = sentences[~((sentences.index.isin(indexes))&(sentences['split']=='train'))].reset_index(drop=True)
print(len(sentences))

963636
706689


In [6]:
data_0 = sentences.loc[sentences['split']=='valid'].reset_index(drop=True)
valid_0 = data_0.sample(frac=0.5, random_state=2000)
train_0 = data_0[~data_0.index.isin(valid_0.index)]

data_1 = sentences.loc[sentences['split']=='train'].reset_index(drop=True)
valid_1 = data_1.sample(frac=0.5, random_state=2000)
train_1 = data_1[~data_1.index.isin(valid_1.index)]

train = pd.concat([train_0, train_1], axis=0).sample(frac=1, random_state=2000).reset_index(drop=True)
valid = pd.concat([valid_0, valid_1], axis=0).sample(frac=1, random_state=2000).reset_index(drop=True)

del data_0, data_1, valid_0, valid_1, train_0, train_1
all_ids = sentences['id'].to_list()
train_ids = train['id'].to_list()
valid_ids = valid['id'].to_list()

# in kaggle notebook, validating is very time-consuming, so here I use a very small validation set, rather than 5667.
valid = valid.sample(n=2000, random_state=42)

print(len(all_ids))
print(len(train_ids))
print(len(valid_ids))

706689
285635
285634


In [7]:
func = lambda x : os.path.isfile(f'/content/drive/MyDrive/audio/dataset/train_mp3s/{x}.mp3')
train["check"] = train["id"].apply(func)
valid["check"] = valid["id"].apply(func)

In [8]:
train = train[train["check"]]
valid = valid[valid["check"]]
print(train.shape, valid.shape)

(13993, 5) (100, 5)


In [9]:
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

In [10]:
del all_ids, train_ids, valid_ids
gc.collect()

34

# 여기부턴 모듈 테스트 하기

# dataset 불러와 지는거 확인

In [ ]:
import whisper_dataset
import whisper_model

In [ ]:
model, processor = whisper_model.load_model()

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
del model, processor
import gc
gc.collect()

In [ ]:
loader = whisper_dataset.WhisperDataset(train, processor)

In [ ]:
d = next(iter(loader))

(1, 102528)
{'input_features': array([[-0.6818743, -0.6818743, -0.6818743, ..., -0.6818743, -0.6818743,
        -0.6818743],
       [-0.6818743, -0.6818743, -0.6818743, ..., -0.6818743, -0.6818743,
        -0.6818743],
       [-0.6818743, -0.6818743, -0.6818743, ..., -0.6818743, -0.6818743,
        -0.6818743],
       ...,
       [-0.6818743, -0.6818743, -0.6818743, ..., -0.6818743, -0.6818743,
        -0.6818743],
       [-0.6818743, -0.6818743, -0.6818743, ..., -0.6818743, -0.6818743,
        -0.6818743],
       [-0.6818743, -0.6818743, -0.6818743, ..., -0.6818743, -0.6818743,
        -0.6818743]], dtype=float32), 'labels': [50258, 50363, 29045, 255, 29045, 122, 29045, 110, 156, 100, 233, 220, 29045, 243, 156, 12811, 29045, 122, 156, 12811, 220, 29045, 97, 29045, 122, 156, 100, 250, 29045, 101, 29045, 122, 220, 29045, 98, 156, 100, 229, 29045, 243, 156, 100, 229, 29045, 229, 220, 29045, 117, 156, 100, 253, 29045, 97, 156, 100, 233, 220, 29045, 243, 29045, 123, 29045, 249, 156, 25787,

In [ ]:
data_collator = whisper_dataset.DataCollatorCTCWithPadding(processor)

In [ ]:
data_collator([d])

torch.Size([1, 80, 3000])
torch.Size([1, 128])


{'input_features': tensor([[[-0.6819, -0.6819, -0.6819,  ..., -0.6819, -0.6819, -0.6819],
         [-0.6819, -0.6819, -0.6819,  ..., -0.6819, -0.6819, -0.6819],
         [-0.6819, -0.6819, -0.6819,  ..., -0.6819, -0.6819, -0.6819],
         ...,
         [-0.6819, -0.6819, -0.6819,  ..., -0.6819, -0.6819, -0.6819],
         [-0.6819, -0.6819, -0.6819,  ..., -0.6819, -0.6819, -0.6819],
         [-0.6819, -0.6819, -0.6819,  ..., -0.6819, -0.6819, -0.6819]]]), 'labels': tensor([[50258, 50363, 29045,   255, 29045,   122, 29045,   110,   156,   100,
           233,   220, 29045,   243,   156, 12811, 29045,   122,   156, 12811,
           220, 29045,    97, 29045,   122,   156,   100,   250, 29045,   101,
         29045,   122,   220, 29045,    98,   156,   100,   229, 29045,   243,
           156,   100,   229, 29045,   229,   220, 29045,   117,   156,   100,
           253, 29045,    97,   156,   100,   233,   220, 29045,   243, 29045,
           123, 29045,   249,   156, 25787,   220,   1

# 모델 불러와보기

In [ ]:
import whisper_model

In [ ]:
model = whisper_model.load_model()

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

# 훈련!

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/audio/whisper/")

In [11]:
import train_whisper
# import imp
# imp.reload(train_whisper)

In [ ]:
train_whisper.processor.feature_extractor.nb_max_frames = 3000
train_whisper.processor.feature_extractor.n_samples = 480000

In [12]:
whisper_asr = train_whisper.whisper_asr(train, valid)

loading model...


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/278 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

loading dataset...
Done!


In [13]:
whisper_asr.train()

training... !!!!!!


Step,Training Loss,Validation Loss,Wer
100,1.134900,0.583931,0.437173
200,0.735500,0.782883,0.571990
300,0.884100,1.014079,0.729058
400,1.083000,1.235811,0.750000
500,1.206600,1.200033,0.715969
600,1.166600,1.123348,0.721204
700,1.088400,1.131851,0.795812
800,1.164000,1.085855,0.752618


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
